<a href="https://colab.research.google.com/github/GuilhermeAlmanza/real_estate_pred/blob/master/src/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# loop the databases, preprocessing in a function

import keras 
import pandas as pd
import numpy as np
import tensorflow as tf

#load files

abcb4_timedata = pd.read_csv("/content/MRVE3 Dados Históricos.csv", sep=",", index_col=None)
# print(abcb4_timedata)

#preprocessing
abcb4_timedata['Data'] = pd.to_datetime(abcb4_timedata['Data'], format="%d.%m.%Y")

abcb4_timedata['Último'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Último']]
abcb4_timedata['Abertura'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Abertura']]
abcb4_timedata['Máxima'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Máxima']]
abcb4_timedata['Mínima'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Mínima']]

abcb4_timedata['Vol.'] = [float(record.replace("K", "0").replace("M", "0000").replace(",", "")) for record in abcb4_timedata['Vol.']]

abcb4_timedata['Var%'] = [float(record.removesuffix("%").replace(",", "."))/100 for record in abcb4_timedata['Var%']]

df = pd.DataFrame(abcb4_timedata, index=None)
df = df.drop(columns='Data')
print(df)


      Último  Abertura  Máxima  Mínima        Vol.    Var%
0       6.42      6.38    6.52    6.30   2520000.0  0.0000
1       6.42      6.23    6.45    6.16   6780000.0  0.0239
2       6.27      6.16    6.35    6.15   6620000.0  0.0212
3       6.14      6.35    6.45    6.09   8830000.0 -0.0376
4       6.38      6.79    6.86    6.25  17400000.0 -0.0713
...      ...       ...     ...     ...         ...     ...
3882    8.22      8.38    8.38    7.99   2470000.0 -0.0226
3883    8.41      8.38    8.49    8.15   1490000.0 -0.0129
3884    8.52      8.26    8.58    7.99   5010000.0  0.0035
3885    8.49      8.63    8.92    8.37   5730000.0 -0.0151
3886    8.62      8.20    8.81    8.01   7750000.0  0.0448

[3887 rows x 6 columns]


In [ ]:
#Train, Test and Validation groups
## receive as a function (including proportions)

from sklearn.model_selection import train_test_split

label_list_train_test = list(range(df.shape[0]))
data_train, data_test, label_train, label_test = train_test_split(df, label_list_train_test, test_size=0.25, random_state=42, shuffle=False)

# print(data_train, "\n" ,data_test)

data_validation, data_test, label_validation, label_test= train_test_split(data_test, label_test, test_size=0.66, random_state=42, shuffle=False)

print(f"data train: \n{data_train} \ndata test\n: {data_test} \ndata validation\n: {data_validation}")


data train: 
      Último  Abertura  Máxima  Mínima        Vol.    Var%
0       6.42      6.38    6.52    6.30   2520000.0  0.0000
1       6.42      6.23    6.45    6.16   6780000.0  0.0239
2       6.27      6.16    6.35    6.15   6620000.0  0.0212
3       6.14      6.35    6.45    6.09   8830000.0 -0.0376
4       6.38      6.79    6.86    6.25  17400000.0 -0.0713
...      ...       ...     ...     ...         ...     ...
2910   11.78     12.38   12.39   11.78   6020000.0 -0.0313
2911   12.16     12.49   12.59   12.13   3340000.0 -0.0233
2912   12.45     12.93   12.93   12.43   4080000.0 -0.0379
2913   12.94     13.01   13.01   12.79   1630000.0 -0.0069
2914   13.03     12.59   13.13   12.41   4070000.0  0.0457

[2915 rows x 6 columns] 
data test
:       Último  Abertura  Máxima  Mínima       Vol.    Var%
3245   10.73     10.85   11.06   10.73  3280000.0 -0.0028
3246   10.76     10.87   11.03   10.71  2270000.0 -0.0019
3247   10.78     10.80   10.95   10.75  2660000.0  0.0113
3248   10

In [ ]:
#Model 
## receive params in a function

import numpy as np
from keras import layers
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
import math

def pearson_correlation(data_test_target, predictions):
        
    len_predictions = len(predictions)
    sum_target = sum(data_test_target)
    sum_predictions = sum(predictions)
    sum_ = sum([x * y for x, y in zip(data_test_target, predictions)])
    sum_target_sqd = sum([x ** 2 for x in data_test_target])
    sum_pred_sqd = sum([y ** 2 for y in predictions])
    
    numerator = len_predictions * sum_ - sum_target * sum_predictions
    denominator = math.sqrt((len_predictions * sum_target_sqd - sum_target ** 2) * (len_predictions * sum_pred_sqd - sum_predictions ** 2))
    
    return numerator / denominator

def metrics(data_test_target, predictions):
  mae = round(mean_absolute_error(data_test_target, predictions) ,4)
  mape = round(np.mean(np.abs((data_test_target - predictions) / data_test_target)) * 100, 4)
  mse = round(mean_squared_error(data_test_target, predictions), 4)
  rmse = round(mse**(0.5), 4)
  rmspe = round((np.sqrt(np.mean(np.square((data_test_target - predictions) / data_test_target)))) * 100, 4)
  r2 = round(r2_score(data_test_target, predictions), 4)
  pearson  = pearsonr(data_test_target, predictions)
  return mae, mape, mse, rmse, rmspe, r2, pearson

def modelLstm(epochs:int, batch_size:int):

  num_timesteps = 1

  #target data
  data_train_target = (data_train.loc[:,"Último"]).values
  data_train.drop(columns = 'Último')

  data_test_target = (data_test.loc[:,"Último"]).values
  data_test.drop(columns = 'Último')

  data_validation_target = (data_validation.loc[:,"Último"]).values
  data_validation.drop(columns = 'Último')

  #reshape data
  data_train_in = data_train.values
  data_train_in = data_train_in.reshape(data_train_in.shape[0], num_timesteps, data_train_in.shape[1])

  data_test_in = data_test.values
  data_test_in = data_test_in.reshape(data_test_in.shape[0], num_timesteps, data_test_in.shape[1])

  data_validation_in = data_validation.values
  data_validation_in = data_validation_in.reshape(data_validation_in.shape[0], num_timesteps, data_validation_in.shape[1])

  model = keras.Sequential([
      layers.LSTM(64, input_shape=(num_timesteps, data_train_in.shape[2])), #timesteps, features
      layers.Dense(1)
  ])

  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  history = model.fit(data_train_in, data_train_target, epochs=epochs, batch_size=batch_size, validation_data=(data_validation_in, data_validation_target))
  scores = model.evaluate(data_test_in, data_test_target)
  predictions = model.predict(data_test_in)

  mae, mape, mse, rmse, rmspe, r2, pearson = metrics(data_test_target, predictions)

  # print(f"mae: {mae} \nmape: {mape} \nmse: {mse}, \nrmse: {rmse} \nrmspe: {rmspe} \nr2: {r2} \nstatistic pearson: {statistic_pearson} \npvalue pearson: {pvalue_pearson}")

# pearson correlation
# params variation

def paramsVariation():
  range_epochs = [5, 25, 50, 100]
  range_batch_size = [25, 50, 100, 200]
  results = [modelLstm(epochs, batch) for epochs, batch in zip(range_epochs, range_batch_size)]

paramsVariation()



Epoch 1/5
117/117 [==============================] - 4s 10ms/step - loss: 138.0172 - mae: 11.2028 - val_loss: 151.0922 - val_mae: 12.1569
Epoch 2/5
117/117 [==============================] - 1s 4ms/step - loss: 100.1025 - mae: 9.3655 - val_loss: 111.4676 - val_mae: 10.4003
Epoch 3/5
117/117 [==============================] - 1s 5ms/step - loss: 71.5986 - mae: 7.6950 - val_loss: 81.1530 - val_mae: 8.8233
Epoch 4/5
117/117 [==============================] - 1s 4ms/step - loss: 50.8731 - mae: 6.2114 - val_loss: 58.5211 - val_mae: 7.4309
Epoch 5/5
21/21 [==============================] - 1s 4ms/step
<class 'scipy.stats._stats_py.PearsonRResult'>


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Epoch 1/25
59/59 [==============================] - 3s 14ms/step - loss: 142.6700 - mae: 11.4239 - val_loss: 172.4831 - val_mae: 13.0070
Epoch 2/25
59/59 [==============================] - 0s 4ms/step - loss: 128.6675 - mae: 10.7926 - val_loss: 156.6820 - val_mae: 12.3847
Epoch 3/25
59/59 [==============================] - 0s 5ms/step - loss: 115.8149 - mae: 10.1793 - val_loss: 142.0832 - val_mae: 11.7805
Epoch 4/25
59/59 [==============================] - 0s 5ms/step - loss: 104.0606 - mae: 9.5854 - val_loss: 128.6579 - val_mae: 11.1962
Epoch 5/25
59/59 [==============================] - 0s 5ms/step - loss: 93.3723 - mae: 9.0096 - val_loss: 116.3440 - val_mae: 10.6321
Epoch 6/25
59/59 [==============================] - 0s 5ms/step - loss: 83.6269 - mae: 8.4560 - val_loss: 105.0099 - val_mae: 10.0850
Epoch 7/25
59/59 [==============================] - 0s 5ms/step - loss: 74.8004 - mae: 7.9152 - val_loss: 94.5766 - val_mae: 9.5538
Epoch 8/25
59/59 [==============================] - 0s 5

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Epoch 1/50
30/30 [==============================] - 3s 24ms/step - loss: 128.0177 - mae: 10.7634 - val_loss: 159.2693 - val_mae: 12.4887
Epoch 2/50
30/30 [==============================] - 0s 6ms/step - loss: 120.6662 - mae: 10.4179 - val_loss: 150.8056 - val_mae: 12.1451
Epoch 3/50
30/30 [==============================] - 0s 6ms/step - loss: 113.6622 - mae: 10.0750 - val_loss: 142.6613 - val_mae: 11.8050
Epoch 4/50
30/30 [==============================] - 0s 6ms/step - loss: 106.9749 - mae: 9.7384 - val_loss: 134.9041 - val_mae: 11.4718
Epoch 5/50
30/30 [==============================] - 0s 6ms/step - loss: 100.6415 - mae: 9.4078 - val_loss: 127.4947 - val_mae: 11.1442
Epoch 6/50
30/30 [==============================] - 0s 7ms/step - loss: 94.6437 - mae: 9.0822 - val_loss: 120.4153 - val_mae: 10.8219
Epoch 7/50
30/30 [==============================] - 0s 6ms/step - loss: 88.9587 - mae: 8.7645 - val_loss: 113.7167 - val_mae: 10.5078
Epoch 8/50
30/30 [==============================] - 0

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Epoch 1/100
15/15 [==============================] - 3s 46ms/step - loss: 111.7613 - mae: 9.9817 - val_loss: 141.2658 - val_mae: 11.7458
Epoch 2/100
15/15 [==============================] - 0s 8ms/step - loss: 106.5313 - mae: 9.7156 - val_loss: 135.1156 - val_mae: 11.4810
Epoch 3/100
15/15 [==============================] - 0s 9ms/step - loss: 101.4827 - mae: 9.4523 - val_loss: 129.1777 - val_mae: 11.2194
Epoch 4/100
15/15 [==============================] - 0s 7ms/step - loss: 96.6373 - mae: 9.1923 - val_loss: 123.4547 - val_mae: 10.9614
Epoch 5/100
15/15 [==============================] - 0s 7ms/step - loss: 91.9934 - mae: 8.9359 - val_loss: 117.9412 - val_mae: 10.7070
Epoch 6/100
11/15 [=====================>........] - ETA: 0s - loss: 87.5591 - mae: 8.6912

KeyboardInterrupt: ignored